In [2]:
# import stanza

# stanza.download('en') # download English model
# nlp = stanza.Pipeline('en') # initialize English neural pipeline
# doc = nlp("Barack Obama was born in Hawaii.")

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [4]:
file_path = "data/accepted_2007_to_2018Q4.csv"
# data = pd.read_csv(file_path)
# data

In [5]:
from src.data.features import select_features, features, DataType


df = pd.read_csv(file_path)
df = select_features(df)

/var/folders/dv/nzlg6jm168311nq7f93wfrd80000gn/T/ipykernel_85822/2249987806.py:4: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [6]:
# df

In [7]:
num_features = [ ft for ft, type in features.items() if type in [DataType.NUMERICAL, DataType.NUMERICAL_LOG] ]
cat_features = [ ft for ft, type in features.items() if type in [DataType.CATEGORICAL] and not ft == "loan_status"]
# num_features.remove("emp_length")
# cat_features.append("emp_length")

In [8]:
cat_features

['term', 'purpose', 'grade', 'home_ownership', 'verification_status']

In [19]:
df_2 = df.copy()

In [25]:
one_hot = pd.get_dummies(df_2, columns=cat_features, drop_first=True)

In [32]:
one_hot.drop(columns="desc", inplace=True)

In [22]:
df_2 = df_2.join(one_hot)

In [12]:
df_num_paidoff = df[df["loan_status"].apply(lambda x: "Fully Paid" == x)][num_features]
df_num_default = df[df["loan_status"].apply(lambda x: "Charged Off" == x)][num_features]

In [13]:
# df2 = df[df["loan_status"].apply(lambda x: x in ["Fully Paid", "Charged Off"])]
# df2["num_loan_status"] = df["loan_status"].apply(lambda x: 1 if x == "Fully Paid" else 0)

In [29]:
from sklearn.model_selection import train_test_split

def split_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    ftrs = list(df.columns)
    # ftrs = list(features.copy().keys())
    ftrs.remove("loan_status")
    X, y = df[ftrs], df[["loan_status"]]
    X_train, X_devtest, y_train, y_devtest = train_test_split(X, y, test_size=.2)
    X_dev, X_test, y_dev, y_test = train_test_split(X_devtest, y_devtest, test_size=.5)

    return X_train, X_dev, X_test, y_train, y_dev, y_test

In [33]:
X_train, X_dev, X_test, y_train, y_dev, y_test = split_data(one_hot)

In [45]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train, y_train)

In [47]:
y_train_resampled.mean()

loan_status    0.337011
dtype: float64

In [ ]:
from scipy import stats

pbc = df[num_features].corrwith(df["loan_status"].astype("float"), method=stats.pointbiserialr)

In [ ]:
pbc.T

,0,1
log_loan_amnt,-0.057460,9.393343e-52
int_rate,-0.216722,0.000000e+00
fico,0.125337,1.577049e-240
inq_last_6mths,-0.068211,3.121620e-72
revol_util,-0.077224,4.143614e-92
delinq_2yrs,-0.013011,6.156438e-04
pub_rec,-0.022042,6.548254e-09
open_acc,-0.022078,6.189215e-09
revol_inc_rat,-0.032577,9.774263e-18
total_acc,0.006954,6.718634e-02


In [ ]:
df_num_default.describe().loc[['mean', "50%", 'std']].T

,mean,50%,std
log_loan_amnt,9.446931,9.574983,0.648319
int_rate,15.462266,15.310000,4.267321
fico,696.006009,692.000000,28.278211
inq_last_6mths,0.998719,1.000000,1.122145
revol_util,60.206618,63.400000,24.128454
delinq_2yrs,0.211506,0.000000,0.659668
pub_rec,0.088366,0.000000,0.330656
open_acc,10.816767,10.000000,4.713373
revol_inc_rat,2.935372,2.496000,2.289760
total_acc,23.681214,22.000000,11.110526


In [ ]:
df_num_paidoff.describe().loc[['mean', "50%", 'std']].T

,mean,50%,std
log_loan_amnt,9.340635,9.392662,0.653947
int_rate,12.836954,12.530000,4.167301
fico,707.795569,702.000000,33.746871
inq_last_6mths,0.796245,0.000000,1.033898
revol_util,54.680830,57.200000,25.413690
delinq_2yrs,0.189006,0.000000,0.602843
pub_rec,0.069767,0.000000,0.292407
open_acc,10.530655,10.000000,4.558833
revol_inc_rat,2.731709,2.294400,2.195640
total_acc,23.900550,22.000000,11.160515


In [ ]:
len(df_num_default) + len(df_num_paidoff)

69276

In [ ]:
df[num_features].corr()

,log_loan_amnt,int_rate,fico,inq_last_6mths,revol_util,delinq_2yrs,pub_rec,open_acc,revol_inc_rat,total_acc,credit_age,log_annual_inc,dti,desc_len
log_loan_amnt,1.000000,0.224906,0.057203,-0.032318,0.118081,-0.001148,-0.030638,0.213630,0.162013,0.260202,0.220920,0.465158,0.110731,-0.020701
int_rate,0.224906,1.000000,-0.642284,0.191317,0.427106,0.131019,0.087361,0.078581,0.057175,0.029023,-0.047764,0.026330,0.184327,-0.034785
fico,0.057203,-0.642284,1.000000,-0.065675,-0.563469,-0.187738,-0.147352,-0.069124,-0.084547,0.029040,0.135663,0.090561,-0.197793,0.068002
inq_last_6mths,-0.032318,0.191317,-0.065675,1.000000,-0.082092,0.015529,0.029718,0.092815,-0.051719,0.110310,-0.020558,0.058733,-0.015858,0.050865
revol_util,0.118081,0.427106,-0.563469,-0.082092,1.000000,-0.007730,0.004898,-0.057687,0.311726,-0.039173,0.017996,0.055740,0.274361,-0.049250
delinq_2yrs,-0.001148,0.131019,-0.187738,0.015529,-0.007730,1.000000,-0.010161,0.048142,-0.077601,0.117654,0.096530,0.070187,0.000816,-0.022918
pub_rec,-0.030638,0.087361,-0.147352,0.029718,0.004898,-0.010161,1.000000,-0.010776,-0.081541,-0.013952,0.061882,-0.009018,-0.031216,-0.028023
open_acc,0.213630,0.078581,-0.069124,0.092815,-0.057687,0.048142,-0.010776,1.000000,0.186439,0.674506,0.197921,0.256956,0.338810,-0.063155
revol_inc_rat,0.162013,0.057175,-0.084547,-0.051719,0.311726,-0.077601,-0.081541,0.186439,1.000000,0.129786,0.167099,-0.081393,0.362061,-0.018999
total_acc,0.260202,0.029023,0.029040,0.110310,-0.039173,0.117654,-0.013952,0.674506,0.129786,1.000000,0.344419,0.375938,0.262267,-0.047811


In [ ]:
df["loan_status"].value_counts(df["loan_status"].unique)

loan_status
Fully Paid                                             0.832946
Charged Off                                            0.143006
Does not meet the credit policy. Status:Fully Paid     0.017159
Does not meet the credit policy. Status:Charged Off    0.006889
Name: proportion, dtype: float64